In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=42ec5e55539b78f6acb933a7316c42b1e92f1b754fb5d581dc0085a9a5cbdd63
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58747 sha256=4fd7ed140ae0cafb502b2d8cc5c834e0442e0099a6442413820fcb603f0c0051
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [22]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *
from datetime import date, datetime
import spark

In [28]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()

In [30]:
def log(log_date,error_desc, log_text, row_count):
  schema= StructType([StructField("log_date",StringType(),True),
                      StructField("status",StringType(), True),
                      StructField("log_text", StringType(), True),
                      StructField("row_count",IntegerType(), True),
  ])
  success_list =[(log_date,error_desc,log_text,row_count)]
  logdf = spark.createDataFrame(success_list,schema)
  current_date = str(date.today())
  current_timestamp = str(datetime.strftime(datetime.now(),"%Y%m%d%H%M%S"))
  #logdf.show()
  logdf.coalesce(1).write.mode('append').csv('/content/logfile.csv')

In [34]:
#log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Success','Data read test',1)

DataFrame[log_date: string, status: string, log_text: string, row_count: int]

In [64]:
logdata = spark.read.csv('/content/logfile.csv')
logdata.show()

+-------------------+-------+--------------------+----+
|                _c0|    _c1|                 _c2| _c3|
+-------------------+-------+--------------------+----+
|2023-07-10 08:24:03|Success|Dataset 1 filtere...| 500|
|2023-07-10 07:43:07|Success|Dataset 2 read su...|1000|
|2023-07-10 07:43:05|Success|Dataset 1 read su...|1000|
|2023-07-10 08:27:36|Success|   Rename successful|   1|
|2023-07-10 08:27:12|Success|   Rename successful|   1|
|2023-07-10 07:40:24|Success|      Data read test|   1|
|2023-07-10 07:40:42|Success|      Data read test|   1|
+-------------------+-------+--------------------+----+



In [31]:
# three parameters as per requirement
# parameters can be passed through dbutils

# dbutils.widgets.text('ds1path')
# ds1_path = dbutils.widgets.get('ds1path)
ds1_path='/content/dataset_one.csv'
ds2_path='/content/dataset_two.csv'
country_names = ['United Kingdom', 'Netherlands']

In [35]:
# Read dataset_one and dataset_two based on the paths provided

try:
  ds1 = spark.read.csv(ds1_path, header=True)
  ds1.show()
  # ds1 has country
  ds2 = spark.read.csv(ds2_path, header=True)
  ds2.show()
  # ds2 financial details
  log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Success','Dataset 1 read successful',ds1.count())
  log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Success','Dataset 2 read successful',ds2.count())
except Exception as e:
  log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Failed',e.args,0)


+---+----------+----------+--------------------+--------------+
| id|first_name| last_name|               email|       country|
+---+----------+----------+--------------------+--------------+
|  1|    Feliza|    Eusden|  feusden0@ameblo.jp|        France|
|  2| Priscilla|   Le Pine|plepine1@biglobe....|        France|
|  3|    Jaimie|    Sandes|jsandes2@reuters.com|        France|
|  4|      Nari|   Dolphin|ndolphin3@cbsloca...|        France|
|  5|     Garik|     Farre|gfarre4@economist...|        France|
|  6|   Kordula|   Broodes| kbroodes5@amazon.de|        France|
|  7|     Rakel|   Ingliby|    ringliby6@ft.com| United States|
|  8|      Derk| Mattielli|dmattielli7@slide...| United States|
|  9|    Karrah|   Boshard|   kboshard8@ihg.com|        France|
| 10| Friedrich|  Kreutzer|fkreutzer9@busine...|        France|
| 11|      Conn|   Claiden| cclaidena@vimeo.com|        France|
| 12|     Karel|   Crippin| kcrippinb@google.pl|        France|
| 13| Millisent|     Joint| mjointc@stat

DataFrame[log_date: string, status: string, log_text: string, row_count: int]

DataFrame[log_date: string, status: string, log_text: string, row_count: int]

In [51]:
# generic function to filter based on country or list of countries

def to_filter(df, country_name):
  df_filtered = df.filter(df['country'].contains(country_name))
  return df_filtered



In [38]:
# generic function to rename column name to be readable

def rename(df,old_name,new_name):
  df_renamed = df.withColumnRenamed(old_name,new_name)
  return df_renamed


In [48]:
t = ds1.filter(ds1['id']=='0')
t.show()

+---+----------+---------+-----+-------+
| id|first_name|last_name|email|country|
+---+----------+---------+-----+-------+
+---+----------+---------+-----+-------+



In [61]:
try:
  for country in country_names:
    print(country)
    f = to_filter(ds1, country)
    f.show()
    t = t.union(f)
  t.show()
  log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Success','Dataset 1 filtered on country successful',t.count())
except Exception as e:
  log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Failed',e.args,0)

United Kingdom
+---+----------+-----------------+--------------------+--------------+
| id|first_name|        last_name|               email|       country|
+---+----------+-----------------+--------------------+--------------+
| 18|   Richard|           Drinan|rdrinanh@odnoklas...|United Kingdom|
| 32|    Wallis|          Bamford|wbamfordv@t-onlin...|United Kingdom|
| 33|   Saundra|          Westall|swestallw@blinkli...|United Kingdom|
| 34|     Ewart|        Rosengren|erosengrenx@usato...|United Kingdom|
| 67|     Lorry|          Beavors|lbeavors1u@techno...|United Kingdom|
| 70|  Freedman|           Dresse|fdresse1x@bloglin...|United Kingdom|
| 91|      Audy|           Eplate|aeplate2i@webeden...|United Kingdom|
|177|   Alfredo|          Wissbey|awissbey4w@geocit...|United Kingdom|
|197|    Penrod|          Volette|  pvolette5g@ask.com|United Kingdom|
|199|   Ermanno|         Surcombe|esurcombe5i@bigca...|United Kingdom|
|294|    Lindie|         MacAskie|lmacaskie85@sina....|United 

DataFrame[log_date: string, status: string, log_text: string, row_count: int]

In [60]:
# datasets joined on id column - type: inner
# ds is joined dataset with filtered dataset one
try:
  ds = t.join(ds2, how='inner', on='id')
  ds.show()
  log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Success','Join successful',ds.count())
except Exception as e:
  log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Failed',e.args,0)

+---+----------+-----------------+--------------------+--------------+--------------------+--------------------+------------------+
| id|first_name|        last_name|               email|       country|               btc_a|                cc_t|              cc_n|
+---+----------+-----------------+--------------------+--------------+--------------------+--------------------+------------------+
| 18|   Richard|           Drinan|rdrinanh@odnoklas...|United Kingdom|1ErM8yuF3ytzzxLy1...|      china-unionpay| 56022230876188334|
| 32|    Wallis|          Bamford|wbamfordv@t-onlin...|United Kingdom|12sxmYnPcADAXw1Yk...|             maestro| 50387077934280351|
| 33|   Saundra|          Westall|swestallw@blinkli...|United Kingdom|1GZ7QB7GUFSWnkBHm...|          mastercard|  5363176326162847|
| 34|     Ewart|        Rosengren|erosengrenx@usato...|United Kingdom|12o8zrHx6snCPbtko...|       visa-electron|  4175005704373695|
| 67|     Lorry|          Beavors|lbeavors1u@techno...|United Kingdom|12ya1E

In [63]:
# renaming columns

try:
  r1=rename(ds,'btc_a','bitcoin_address')
  r2=rename(r1,'cc_t','credit_card_type')
  r3=rename(r2,'cc_n','credit_card_number')
  r3.show()
  log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Success','Rename successful',1)
except Exception as e:
  log(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),'Failed',e.args,0)


+---+----------+-----------------+--------------------+--------------+--------------------+--------------------+------------------+
| id|first_name|        last_name|               email|       country|     bitcoin_address|    credit_card_type|credit_card_number|
+---+----------+-----------------+--------------------+--------------+--------------------+--------------------+------------------+
| 18|   Richard|           Drinan|rdrinanh@odnoklas...|United Kingdom|1ErM8yuF3ytzzxLy1...|      china-unionpay| 56022230876188334|
| 32|    Wallis|          Bamford|wbamfordv@t-onlin...|United Kingdom|12sxmYnPcADAXw1Yk...|             maestro| 50387077934280351|
| 33|   Saundra|          Westall|swestallw@blinkli...|United Kingdom|1GZ7QB7GUFSWnkBHm...|          mastercard|  5363176326162847|
| 34|     Ewart|        Rosengren|erosengrenx@usato...|United Kingdom|12o8zrHx6snCPbtko...|       visa-electron|  4175005704373695|
| 67|     Lorry|          Beavors|lbeavors1u@techno...|United Kingdom|12ya1E

DataFrame[log_date: string, status: string, log_text: string, row_count: int]

In [86]:
# selecting required columns

final_ds = r3.select('email','country','bitcoin_address','credit_card_type')
final_ds.show()

+--------------------+--------------+--------------------+--------------------+
|               email|       country|     bitcoin_address|    credit_card_type|
+--------------------+--------------+--------------------+--------------------+
|rdrinanh@odnoklas...|United Kingdom|1ErM8yuF3ytzzxLy1...|      china-unionpay|
|wbamfordv@t-onlin...|United Kingdom|12sxmYnPcADAXw1Yk...|             maestro|
|swestallw@blinkli...|United Kingdom|1GZ7QB7GUFSWnkBHm...|          mastercard|
|erosengrenx@usato...|United Kingdom|12o8zrHx6snCPbtko...|       visa-electron|
|lbeavors1u@techno...|United Kingdom|12ya1ED93ApPBQRSC...|            bankcard|
|fdresse1x@bloglin...|United Kingdom|1F8zXTEaf4AFpztMN...|          mastercard|
|aeplate2i@webeden...|United Kingdom|19MZSy1H8S4SaXsmS...|diners-club-carte...|
|awissbey4w@geocit...|United Kingdom|1AJzqEgbbFh2TNLFZ...|     americanexpress|
|  pvolette5g@ask.com|United Kingdom|1Q6UV84patYXfzEdA...| diners-club-enroute|
|esurcombe5i@bigca...|United Kingdom|1GY